In [2]:
from tool import *
import plotly.express as px
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
import pyautogui

## GSCP Sales PSI Raw data를  DB에 저장

In [13]:
filepath = get_filename()
created_at = '2024030514'
planweek = get_weekname(datetime.datetime(2024, 3, 4))
ver = 'Final'

In [14]:
psi = pd.read_excel(filepath, sheet_name=0, keep_default_na=False, na_values=['', 'nan']) # GSCP로부터 금주 SP를 데이타프레임화trim_column_name(sp)
c1 = psi['Rep From Site'].str.contains('BOEVT')
c2 = psi['Mapping Model.Suffix'].str.match('[2|3][2|4|5|7][G|B|M][A-Z]\d{2}') # ODM 모니터 모델인 것만 찾는 정규표현식
psi = psi[~(c1 & ~c2)].reset_index(drop=True) # BOEVT 모델 중 ODM 모니터가 아닌 것만 제외해서 필터링
trim_column_name(psi)
psi = psi[psi['Rep From Site'].notnull()].reset_index(drop=True)
psi.insert(0, 'Created_at', created_at) # PSI 생성일
psi.insert(0, 'Ver', ver) # 기준 주차명 설정
psi.insert(0, 'Planweek', planweek) # 기준 주차명 설정
psi = psi.rename(columns={'Site':'To Site', 'Rep From Site':'From Site'})
psi = psi.set_index(['Planweek', 'Created_at', 'Ver', 'Division', 'From Site', 'Region', 'To Site', 'Mapping Model.Suffix', 'Rep PMS', 'Category'])[get_weeklist(psi)]
psi.columns.name = 'Week Name'
psi = psi.stack().replace({0:np.nan}).dropna().astype(int).reset_index()
psi.rename(columns={0:'QTY'}, inplace=True)

In [15]:
with open('D:/Data/ODM PSI.db', 'rb') as f:
    psi_db = pickle.load(f)

In [16]:
psi_db = pd.concat([psi_db, psi]).reset_index(drop=True)

In [17]:
psi_db.shape

(702823, 12)

In [18]:
with open('D:/Data/ODM PSI.db', 'wb') as f:
    pickle.dump(psi_db, f)

# PSI 분석

In [3]:
with open('D:/Data/ODM PSI.db', 'rb') as f:
    psi_db = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)

In [4]:
sf = psi_db[psi_db['Category'] == 'Sales Forecast [R+F]'].reset_index(drop=True)

### 1) ODM 전체로 볼 떄

In [5]:
ver_table = sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('SP').reset_index()
ver_table.iloc[-30:, ]

,Planweek,Ver,Created_at,QTY
0,2024-01-08(W02),Final,2024010809,994792
1,2024-01-15(W03),Final,2024011509,1045657
2,2024-01-22(W04),Final,20240122,1080224
3,2024-01-29(W05),Final,2024013016,1114778
4,2024-01-29(W05),Main,2024013009,1114778
5,2024-02-05(W06),Final,2024020615,1142048
6,2024-02-05(W06),Main,2024020509,1100299
7,2024-02-12(W07),Final,2024021412,1138828
8,2024-02-12(W07),Main,2024021217,1263227
9,2024-02-19(W08),Final,2024022112,1193441


In [11]:
i1 = 11
i2 = 9

p_wk = sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :]['Planweek']
ver = sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :]['Ver']

sf1 = sf.merge(sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i1], :3])
sf2 = sf.merge(sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i2], :3])
sf2 = sf2[sf2['Week Name'] >= sf1['Week Name'].min()]

In [12]:
sf_all = pd.concat([sf1, sf2])
sf_all['Month'] = sf_all['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

In [13]:
sf_all = sf_all[sf_all['Month'] >= f"{datetime.date.fromisoformat(p_wk[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(p_wk[:10])):02d}"] # 이번달부터 필터링
sf_all = sf_all[sf_all['Month'] <= '2024-05'] # 24년 4월까지만 한정

In [16]:
# sf_all.insert(7, 'Series', sf_all['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
# sf_all.insert(7, 'Inch', sf_all['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sf_all = pd.merge(sf_all, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [17]:
# IT ODM 모델명에서 시리즈명 추출
c1 = sf_all['From Site'].str.contains('AUO|TPV|TCL|BOEVT') # 모니터 ODM 일 경우
c2 = sf_all['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sf_all['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우

sf_all.loc[c1, 'Series'] = sf_all[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출
sf_all.loc[c2, 'Series'] = sf_all[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sf_all.loc[c3, 'Series'] = 'CL600'

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20484\2109933073.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  sf_all.loc[c2, 'Series'] = sf_all[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [18]:
sf_all = pd.merge(sf_all, site[['To Site', 'Subsidiary', 'Country']], how='left')
sf_all.loc[sf_all['Division']=='PC', 'SCM'] = sf_all.loc[sf_all['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='PC', 'MKT'] = sf_all.loc[sf_all['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='MNT', 'SCM'] = sf_all.loc[sf_all['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='MNT', 'MKT'] = sf_all.loc[sf_all['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
us_b2b = sf_all.loc[sf_all['To Site'].str.contains('ENUS') & (sf_all['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['To Site'].str.contains('ENUS') & (sf_all['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b
sf_all = sf_all.fillna('N/A')

In [20]:
gap = sf_all.groupby(['Region', 'Country', 'MKT', 'SCM', 'Planweek', 'Created_at', 'To Site', 'Ver', 'Series', 'Month', 'Week Name'])['QTY'].sum().unstack(['Planweek', 'Ver', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap = gap.unstack('Month').fillna(0).astype(int)
gap['Sum of Gap'] = gap['Gap'].sum(axis=1)
gap = gap.reset_index()

In [21]:
gap

Planweek   Region  Country  MKT  SCM        To Site  Series        Week Name  \
Ver                                                                            
Created_at                                                                     
Month                                                                          
0             CIS   Russia  이기현  오석현  ERRA_MINSK_DO      22  2024-03-11(W11)   
1             CIS   Russia  이기현  오석현  ERRA_MINSK_DO      22  2024-04-01(W14)   
2             CIS   Russia  이기현  오석현  ERRA_MINSK_DO      22  2024-04-15(W16)   
3             CIS   Russia  이기현  오석현  ERRA_MINSK_DO      24  2024-03-11(W11)   
4             CIS   Russia  이기현  오석현  ERRA_MINSK_DO      24  2024-04-01(W14)   
...           ...      ...  ...  ...            ...     ...              ...   
1910          SEA  Vietnam  박진아  이종혁        EAVH_HN  14T90R  2024-03-25(W13)   
1911          SEA  Vietnam  박진아  이종혁        EAVH_HN  14T90R  2024-04-08(W15)   
1912          SEA  Vietnam  박진아  이종혁        EAVH_HN  14T90R  2024-04-15(W16)   
1913          SEA  Vietnam  박진아  이종혁        EAVH_HN  14T90R  2024-05-06(W19)   
1914          SEA  Vietnam  박진아  이종혁        EAVH_HN  14T90R  2024-05-13(W20)   

Planweek   2024-02-19(W08)                         2024-02-26(W09)          \
Ver                  Final                                    Main           
Created_at      2024022112                              2024022614           
Month              2024-02 2024-03 2024-04 2024-05         2024-02 2024-03   
0                        0     504       0       0               0     504   
1                        0       0     252       0               0       0   
2                        0       0     300       0               0       0   
3                        0    2976       0       0               0    2976   
4                        0       0     864       0               0       0   
...                    ...     ...     ...     ...             ...     ...   
1910                     0       2       0       0               0       2   
1911                     0       0       1       0               0       0   
1912                     0       0       1       0               0       0   
1913                     0       0       0       1               0       0   
1914                     0       0       0       1               0       0   

Planweek                       Gap                         Sum of Gap  
Ver                                                                    
Created_at                                                             
Month      2024-04 2024-05 2024-02 2024-03 2024-04 2024-05             
0                0       0       0       0       0       0          0  
1              252       0       0       0       0       0          0  
2              300       0       0       0       0       0          0  
3                0       0       0       0       0       0          0  
4              864       0       0       0       0       0          0  
...            ...     ...     ...     ...     ...     ...        ...  
1910             0       0       0       0       0       0          0  
1911             1       0       0       0       0       0          0  
1912             2       0       0       0       1       0          1  
1913             0       1       0       0       0       0          0  
1914             0       1       0       0       0       0          0  

[1915 rows x 20 columns]

In [22]:
countries = gap['Country'].unique()

In [27]:
# 국가별 탭으로 엑셀 파일 만들기
df_all = []
for country in countries:
    df = gap[gap['Country'] == country].copy()
    df = df.reset_index(drop=True).set_index(['Region', 'Country', 'To Site', 'Series', 'Week Name'])
    df_all.append(df)

updated_time = datetime.datetime.now().strftime('%Y%m%d%H%M')
filepath = f'D:/SP검토/{updated_time}_{p_wk}_TCL Sales Forecast Review_by_country.xlsx'
with pd.ExcelWriter(filepath) as writer:
    for i, df in enumerate(df_all):
        df.to_excel(writer, sheet_name=f'{countries[i]}', startrow=1)

In [29]:
wb = load_workbook(filepath)

for sheet_name in wb.sheetnames:
    confirm_wk = '2024-03-18(W12)'
    color = PatternFill(fgColor='9999FF', fill_type='solid') # violet
    confirm_color = PatternFill(fgColor='00cc00', fill_type='solid') # Honeydew
    pseudo_confirm_color = PatternFill(fgColor='9ACD32', fill_type='solid') # Yellowgreen
    border_style = Side(border_style="thin", color="000000")
    border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)
    sh = wb[sheet_name]
    sh['G1'] = 'Sub TTL'
    for i in range(8, sh.max_column+1):
        col_letter = get_column_letter(i)
        sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
        sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=2, min_col=5, max_row=5, max_col=sh.max_column):
        for cell in row:
            cell.fill = color
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    merge_range = f'{get_column_letter(sh.max_column)}2:{get_column_letter(sh.max_column)}5'
    sh.merge_cells(merge_range)
    for cell in sh[merge_range][0]:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    for row in sh.iter_rows(min_row=6, min_col=1, max_row=6, max_col=7):
        for cell in row:
            cell.fill = color
    for row in sh.iter_rows(min_row=7, min_col=7, max_row=sh.max_row, max_col=sh.max_column):
        for cell in row:
            cell.border = border
            cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
#     for row in sh.iter_rows(min_row=7, min_col=6):
#         if row[0].value == confirm_wk:
#             for cell in row:
#                 cell.fill = confirm_color
#         elif row[0].value in [get_weekname_from(confirm_wk, i) for i in range(1, 3)]:
#             for cell in row:
#                 cell.fill = pseudo_confirm_color
    sh.freeze_panes = 'H7'
    sh.column_dimensions['B'].width = 15
    sh.column_dimensions['C'].width = 13
    sh.column_dimensions['E'].width = 17.5
    sh.sheet_view.showGridLines = False

In [30]:
wb.save(filepath)
wb.close()

### 2) 업체별로 볼 때

In [11]:
ver_table = sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('SP').reset_index()
ver_table = ver_table[ver_table['From Site'].str.contains('TCL')]
ver_table.iloc[-30:, ]

,Planweek,From Site,Ver,Created_at,QTY
3,2024-01-08(W02),OS_TCL_CN_P,Final,2024010809,369842
9,2024-01-15(W03),OS_TCL_CN_P,Final,2024011509,400397
15,2024-01-22(W04),OS_TCL_CN_P,Final,20240122,414732
24,2024-01-29(W05),OS_TCL_CN_P,Final,2024013016,436568
25,2024-01-29(W05),OS_TCL_CN_P,Main,2024013009,436568
33,2024-02-05(W06),OS_TCL_CN_P,Main,2024020509,446979


In [12]:
i1 = 33
i2 = 24

p_wk = sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Planweek']
vendor = sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['From Site'].split('_')[1]
ver = sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Ver']

In [16]:
sf1 = sf.merge(sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i1], :4])
sf2 = sf.merge(sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i2], :4])
sf2 = sf2[sf2['Week Name'] >= sf1['Week Name'].min()]

In [17]:
sf_all = pd.concat([sf1, sf2])
sf_all['Month'] = sf_all['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

In [18]:
sf_all = sf_all[sf_all['Week Name'] >= [get_weekname_from(p_wk, i) for i in range(-3, 1)][0]] # 이번주 기준 3주전부터 Sorting
sf_all = sf_all[sf_all['Month'] <= '2024-05'] # 24년 4월까지만 한정

In [19]:
sf_all.insert(7, 'Series', sf_all['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
sf_all.insert(7, 'Inch', sf_all['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sf_all = pd.merge(sf_all, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [20]:
sf_all.loc[sf_all['Division']=='PC', 'SCM'] = sf_all.loc[sf_all['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='PC', 'MKT'] = sf_all.loc[sf_all['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='MNT', 'SCM'] = sf_all.loc[sf_all['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='MNT', 'MKT'] = sf_all.loc[sf_all['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
us_b2b = sf_all.loc[sf_all['To Site'].str.contains('ENUS') & (sf_all['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
sf_all.loc[sf_all['To Site'].str.contains('ENUS') & (sf_all['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b
sf_all = sf_all.fillna('N/A')

In [21]:
gap = sf_all.groupby(['Country', 'MKT', 'SCM', 'Planweek', 'Created_at', 'To Site', 'Ver', 'Series', 'Month', 'Week Name'])['QTY'].sum().unstack(['Planweek', 'Ver', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap = gap.unstack('Month').fillna(0).astype(int)
gap['Sum of Gap'] = gap['Gap'].sum(axis=1)
gap = gap.reset_index()

In [22]:
countries = gap['Country'].unique()

In [34]:
# 국가별 탭으로 엑셀 파일 만들기
df_all = []
for country in countries:
    df = gap[gap['Country'] == country].copy()
    df = df.reset_index(drop=True).set_index(['Country', 'MKT', 'SCM', 'To Site', 'Series', 'Week Name'])
    df_all.append(df)

updated_time = datetime.datetime.now().strftime('%Y%m%d%H%M')
filepath = f'D:/SP검토/{updated_time}_{p_wk}_TCL Sales Forecast Review_by_country.xlsx'
with pd.ExcelWriter(filepath) as writer:
    for i, df in enumerate(df_all):
        df.to_excel(writer, sheet_name=f'{countries[i]}', startrow=1)

## TCL 최근 5주간 변동 현황

In [35]:
ver_table.iloc[-30:, ]

,Planweek,From Site,Ver,Created_at,QTY
3,2024-01-08(W02),OS_TCL_CN_P,Final,2024010809,369842
9,2024-01-15(W03),OS_TCL_CN_P,Final,2024011509,400397
15,2024-01-22(W04),OS_TCL_CN_P,Final,20240122,414732
24,2024-01-29(W05),OS_TCL_CN_P,Final,2024013016,436568
25,2024-01-29(W05),OS_TCL_CN_P,Main,2024013009,436568
33,2024-02-05(W06),OS_TCL_CN_P,Main,2024020509,446979


In [36]:
idxs = [3, 9, 15, 24, 33] # index 넘버를 선택
df = pd.DataFrame()
for i in idxs:
    df = pd.concat([df, sf.merge(sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i], :-1])]).reset_index(drop=True)    

In [37]:
search_period = ['2024-02', '2024-03', '2024-04', '2024-05', '2024-06']

In [38]:
df['Month'] = df['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')
df.insert(7, 'Series', df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
df.insert(7, 'Inch', df['Mapping Model.Suffix'].apply(lambda x:x[:2]))
df = df[df['Month'].isin(search_period)]

df = pd.merge(df, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [39]:
wb = load_workbook(filepath)

for sheet_name in wb.sheetnames:
#     confirm_wk = '2024-02-26(W09)'
    color = PatternFill(fgColor='9999FF', fill_type='solid') # violet
    confirm_color = PatternFill(fgColor='00cc00', fill_type='solid') # Honeydew
    pseudo_confirm_color = PatternFill(fgColor='9ACD32', fill_type='solid') # Yellowgreen
    border_style = Side(border_style="thin", color="000000")
    border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)
    sh = wb[sheet_name]
    sh['F1'] = 'Sub TTL'
    for i in range(7, sh.max_column+1):
        col_letter = get_column_letter(i)
        sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
        sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=2, min_col=6, max_row=5, max_col=sh.max_column):
        for cell in row:
            cell.fill = color
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    merge_range = f'{get_column_letter(sh.max_column)}2:{get_column_letter(sh.max_column)}5'
    sh.merge_cells(merge_range)
    for cell in sh[merge_range][0]:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    for row in sh.iter_rows(min_row=6, min_col=1, max_row=6, max_col=6):
        for cell in row:
            cell.fill = color
    for row in sh.iter_rows(min_row=7, min_col=7, max_row=sh.max_row, max_col=sh.max_column):
        for cell in row:
            cell.border = border
            cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
#     for row in sh.iter_rows(min_row=7, min_col=6):
#         if row[0].value <= confirm_wk:
#             for cell in row:
#                 cell.fill = confirm_color
#         elif row[0].value in [get_weekname_from(confirm_wk, i) for i in range(1, 3)]:
#             for cell in row:
#                 cell.fill = pseudo_confirm_color
    sh.freeze_panes = 'G7'
    sh.column_dimensions['A'].width = 15
    sh.column_dimensions['B'].width = 6
    sh.column_dimensions['C'].width = 6
    sh.column_dimensions['F'].width = 17.5
    sh.sheet_view.showGridLines = False
    
    df_bar = df.groupby(['Country', 'Planweek', 'Series', 'Month']).sum('QTY').loc[sheet_name].reset_index()
    fig = px.bar(df_bar, x='Planweek', y='QTY', color='Month', facet_col='Series', title=f'{sheet_name} Sales Forecast Variation',
                 category_orders={'Month':['2024-02', '2024-03', '2024-04', '2024-05', '2024-06'], 'Planweek':sorted(df['Planweek'].unique())}, width=1200, text_auto=True)
    image_filepath = f'D:/figure/{sheet_name}_weekly_sales_forecast_variation.png'
    fig.write_image(image_filepath)
    
    plot_image = Image(image_filepath)
    sh.add_image(plot_image, f'A{sh.max_row+2}')

wb.save(filepath)
wb.close()